In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
con = sqlite3.connect('amin.db')
pd.read_sql("select * from first_visit", con)

,user_pseudo_id,event_date,first_visit_time,source_name,source_medium,source_origin
0,1742949957.1734750831,2024-12-21,2024-12-21 03:13:52.071116,None,paid,facebook
1,1404665972.1734753464,2024-12-21,2024-12-21 03:57:44.668297,None,paid,facebook
2,28897507.1734786394,2024-12-21,2024-12-21 13:06:35.158577,None,cpc,google
3,132473210.1734772815,2024-12-21,2024-12-21 09:20:15.702832,None,paid,facebook
4,2052533640.1734804643,2024-12-21,2024-12-21 18:10:44.167625,None,paid,facebook
...,...,...,...,...,...,...
539650,1656943616.1755059218,2025-08-13,2025-08-13 04:26:57.941656,Shopping Free Listings,organic,google
539651,2032146311.1755069625,2025-08-13,2025-08-13 07:20:25.526839,Shopping Free Listings,organic,google
539652,2138896961.1755089311,2025-08-13,2025-08-13 12:48:32.223327,Shopping Free Listings,organic,google
539653,2002225121.1755094826,2025-08-13,2025-08-13 14:20:27.366876,Shopping Free Listings,organic,google
